### Practice 1 - Evaluating returns

Markos Flavio B. G. O.

__Context: Risk and return of assets.__

__Course: Portfolio Selection and Risk Management (Coursera, Rice University)__

Create a function that evaluates different types of returns based on a data set of price changes.
    
__Specific objectives__

     1. Evaluate the cumulative, past-average and expected returns from a Data Frame of price changes.

In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy.stats.mstats import gmean

In [163]:
loc = r'D:\Dropbox\_Studies\Current Projects\Stock Market\Coursera\Rice University, Portfolio Selection and Risk Management\Module 1 - Introduction & Risk and Return\data1.csv'
df = pd.read_csv(loc, index_col=0)
print(df.dtypes)
df.head()

APPL       object
WALMART    object
IBM        object
NIKE       object
dtype: object


,APPL,WALMART,IBM,NIKE
1/31/2011,5.20%,3.97%,10.38%,-3.44%
2/28/2011,4.09%,-7.29%,0.32%,7.94%
3/31/2011,-1.33%,0.84%,0.74%,-14.67%
4/29/2011,0.47%,5.63%,4.61%,8.75%
5/31/2011,-0.66%,1.10%,-0.52%,2.59%


In [164]:
df[df.columns] = df[df.columns].replace({'%': ''}, regex=True)
df[df.columns] = df[df.columns].apply(pd.to_numeric)
df.index = pd.to_datetime(df.index)
df.sort_index(ascending=True).head()
df.head()

,APPL,WALMART,IBM,NIKE
2011-01-31,5.20,3.97,10.38,-3.44
2011-02-28,4.09,-7.29,0.32,7.94
2011-03-31,-1.33,0.84,0.74,-14.67
2011-04-29,0.47,5.63,4.61,8.75
2011-05-31,-0.66,1.10,-0.52,2.59


In [1]:
def get_returns(df, time_range=None):
    '''
    Get the cumulative, past-average and expected returns from a dataset of holding period returns.
    
    Parameters:
        df (DataFrame): a DataFrame where the index has type 'datetime64' and columns are percentage changes in price.
        time_range (list): a list with size two, specifying a date range (in format '%d/%m/%Y'). If None, all samples are used.
    Returns:
        df_results (DataFrame): a DataFrame with:
            - The geometric mean of returns for the selected period.
            - The total period return.
            - The expected return.
    '''
    if time_range:
        df = df[(df.index >= datetime.strptime(time_range[0], '%d/%m/%Y')) & (df.index <= datetime.strptime(time_range[1], '%d/%m/%Y'))]
   
    df = df/100 + 1 # getting the gross return
    df_comp = df.apply(np.prod)
    df_geom = df.apply(gmean, axis=0)
    df_exp = df.apply(np.mean)
    
    df_results =  pd.concat([df_comp, df_geom, df_exp], axis=1)
    df_results.columns = ['Cumulative return (%)', 'Average return (%)', 'Expected return (%)']
    df_results = (df_results-1)*100 # getting back the percentages
    return df_results      

In [168]:
dfr = get_returns(df)
dfr

,Cumulative return (%),Average return (%),Expected return (%)
APPL,145.329104,1.506959,1.755167
WALMART,29.140035,0.427122,0.540000
IBM,4.544873,0.074104,0.176833
NIKE,212.602565,1.917761,2.103333


  - The **cumulative return** is the total period return.
  - The **average return** is the constant return that would result in the same cumulative return at the end of the entire period. It's the geometric mean of the available returns. Let's check an example for the APPL symbol:

In [176]:
(1 + dfr.loc['APPL']['Average return (%)']/100)**df.shape[0]

2.4532910441299487

This means that the initial investing amount is multiplied by 2.45, an increase of (2.45*100-1)=145%, which is indeed correct.

  - The **expected return** is the return one would expect to have after a single period, here one month. It's an unbiased estimate of a future return, based on the available data. It's assumed that each data point is an equally likely scenario of what can possibly happen to the price.